In [1]:
import numpy as np
import pandas as pd
#from sklearn.cross_validation import KFold
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit
from XGBoostPackage import xgbClass

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [29]:
class Ensemble(object):
    #base_models=[model1, model2, model3,...]
    def __init__(self, n_folds, stacker, base_models, is_TimeSeries=False):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
        self.S_train=[]
        self.S_test=[]
        self.isTS=is_TimeSeries
        #self_folds=[]
        
    def fit(self, X_train, y_train):
        if not len(np.array(X_train).shape)==0:
            X_train=np.array(X_train)
            y_train=np.array(y_train)
        if not self.isTS:
            kf=KFold(n_splits=self.n_folds, shuffle=True, random_state=17)
        else:
            kf=TimeSeriesSplit(n_splits=self.n_folds)
        self.S_train = np.zeros((X_train.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            #clf=[clfModel for tmp in np.arange(self.n_folds)]
            for j, (train_idx, test_idx) in enumerate(kf.split(X_train)):
                print('startig training model {}, training set {}'.format(i,j))
                X_CVtrain = X_train[train_idx]
                y_CVtrain = y_train[train_idx]
                X_CVholdout = X_train[test_idx]
                # y_CVholdout = y_train[test_idx]
                clf[j].fit(X_CVtrain, y_CVtrain)
                y_pred = clf[j].predict(X_CVholdout)[:]
                self.S_train[test_idx, i] = y_pred
        self.stacker.fit(self.S_train, y_train)
        
    def predict(self, X_test):    
        if not len(np.array(X_test).shape)==0:
            X_test=np.array(X_test)
        self.S_test = np.zeros((X_test.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((X_test.shape[0], self.n_folds))
            for j in range(self.n_folds):
                print('startig predicting model {}, training set {}'.format(i,j))
                S_test_i[:, j] = clf[j].predict(X_test)[:]
            self.S_test[:, i] = S_test_i.mean(1)
        y_pred = self.stacker.predict(self.S_test)[:]
        return y_pred

In [ ]:
class EnsembleClassifier(object):
    #base_models=[model1, model2, model3,...]
    def __init__(self, n_folds, stacker, base_models, is_TimeSeries=False, n_class=1):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
        self.S_train=[]
        self.S_test=[]
        self.isTS=is_TimeSeries
        self.n_class=n_class
        #self_folds=[]
        
    def fit(self, X_train, y_train):
        if not len(np.array(X_train).shape)==0:
            X_train=np.array(X_train)
            y_train=np.array(y_train)
        if not self.isTS:
            kf=KFold(n_splits=self.n_folds, shuffle=True, random_state=17)
        else:
            kf=TimeSeriesSplit(n_splits=self.n_folds)
        self.S_train = np.zeros((X_train.shape[0], len(self.base_models)*self.n_class))
        for i, clf in enumerate(self.base_models):
            #clf=[clfModel for tmp in np.arange(self.n_folds)]
            for j, (train_idx, test_idx) in enumerate(kf.split(X_train)):
                print('startig training model {}, training set {}'.format(i,j))
                X_CVtrain = X_train[train_idx]
                y_CVtrain = y_train[train_idx]
                X_CVholdout = X_train[test_idx]
                # y_CVholdout = y_train[test_idx]
                clf[j].fit(X_CVtrain, y_CVtrain)
                y_pred = clf[j].predict_proba(X_CVholdout)[:]
                self.S_train[test_idx, i:(i+self.n_class)] = y_pred
        self.stacker.fit(self.S_train, y_train)
        
    def predict(self, X_test):    
        if not len(np.array(X_test).shape)==0:
            X_test=np.array(X_test)
        self.S_test = np.zeros((X_test.shape[0], len(self.base_models)*self.n_class))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((X_test.shape[0], self.n_folds))
            for j in range(self.n_folds):
                print('startig predicting model {}, training set {}'.format(i,j))
                S_test_i[:, j] = clf[j].predict_proba(X_test)[:]
            self.S_test[:, i] = S_test_i.mean(1)
        y_pred = self.stacker.predict(self.S_test)[:]
        return y_pred
    
    def predict_proba(self, X_test):    
        if not len(np.array(X_test).shape)==0:
            X_test=np.array(X_test)
        self.S_test = np.zeros((X_test.shape[0], len(self.base_models)*self.n_class))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((X_test.shape[0], self.n_folds))
            for j in range(self.n_folds):
                print('startig predict_proba model {}, training set {}'.format(i,j))
                S_test_i[:, j] = clf[j].predict_proba(X_test)[:]
            self.S_test[:, i] = S_test_i.mean(1)
        y_pred = self.stacker.predict_proba(self.S_test)[:]
        return y_pred

In [23]:
# class Ensemble(object):
#     #base_models=[model1, model2, model3,...]
#     def __init__(self, n_folds, stacker, base_models, is_TimeSeries=False):
#         self.n_folds = n_folds
#         self.stacker = stacker
#         self.base_models = base_models
#         self.fitted_models=[]
#         self.S_train=[]
#         self.S_test=[]
#         self.isTS=is_TimeSeries
#         #self_folds=[]
        
#     def fit(self, X_train, y_train):
#         X_train=np.array(X_train)
#         y_train=np.array(y_train)
#         if not self.isTS:
#             kf=KFold(n_splits=self.n_folds, shuffle=True, random_state=17)
#         else:
#             kf=TimeSeriesSplit(n_splits=self.n_folds)
#         self.S_train = np.zeros((X_train.shape[0], len(self.base_models)))
#         for i, clfModel in enumerate(self.base_models):
#             clf=[clfModel() for tmp in np.arange(self.n_folds)]
#             self.fitted_models.append(clf)
#             for j, (train_idx, test_idx) in enumerate(kf.split(np.arange(len(X_train)))):
#                 X_CVtrain = X_train[train_idx]
#                 y_CVtrain = y_train[train_idx]
#                 X_CVholdout = X_train[test_idx]
#                 # y_CVholdout = y_train[test_idx]
#                 clf[j].fit(X_CVtrain, y_CVtrain)
#                 y_pred = clf[j].predict(X_CVholdout)[:]
#                 self.S_train[test_idx, i] = y_pred
#         self.stacker.fit(self.S_train, y_train)
        
#     def predict(self, X_test):    
#         X_test=np.array(X_test)
#         self.S_test = np.zeros((X_test.shape[0], len(self.base_models)))
#         for i, clfModel in enumerate(self.base_models):
#             clf=self.fitted_models[i]
#             S_test_i = np.zeros((X_test.shape[0], self.n_folds))
#             for j in range(self.n_folds):
#                 S_test_i[:, j] = clf[j].predict(X_test)[:]
#             self.S_test[:, i] = S_test_i.mean(1)
#         y_pred = self.stacker.predict(self.S_test)[:]
#         return y_pred

In [8]:
n=100000
X=pd.DataFrame(np.random.randn(n,1))
y=X.iloc[:,0]+.2*pd.Series(np.random.randn(n))
X_train,y_train=X.iloc[:n/2], y.iloc[:n/2]
X_test, y_test=X.iloc[n/2:], y.iloc[n/2:]



In [6]:
# import kagglegym
# env = kagglegym.make()
# o = env.reset()
# excl = [env.ID_COL_NAME, env.SAMPLE_COL_NAME, env.TARGET_COL_NAME, env.TIME_COL_NAME]
# col = [c for c in o.train.columns if c not in excl]

# O = pd.read_hdf('../input/train.h5')
# d_mean= O[col].median(axis=0)

# ymean_dict = dict(o.train.groupby(["id"])["y"].median())

# X_train=(O[col])[O.timestamp <= 905]
# y_train=O.y[O.timestamp <= 905]
# X_test=(O[col])[O.timestamp > 905]
# y_test=O.y[O.timestamp > 905]
# X_train=X_train.fillna(d_mean)
# X_test=X_test.fillna(d_mean)

KeyboardInterrupt: 

In [30]:
from sklearn.linear_model import LinearRegression, Ridge
n_folds=5
ensembleObj=Ensemble(n_folds=n_folds, stacker=LinearRegression(fit_intercept=False, n_jobs=7), \
                     base_models=[[LinearRegression() for i in np.arange(n_folds)],\
                                 [Ridge() for i in np.arange(n_folds)],\
                                 [xgbClass(objective='reg:linear', eva_metric="rmse") for i in np.arange(n_folds)]], \
                     is_TimeSeries=False)#,\
ensembleObj.fit(X_train=X_train, y_train=y_train)
pred=ensembleObj.predict(X_test=X_test)
r2_score(y_test, pred)
#0.96163045588344376

0.96159169367394837

In [24]:
# from sklearn.linear_model import LinearRegression, Ridge
# n_folds=5
# ensembleObj=Ensemble(n_folds=n_folds, stacker=LinearRegression(fit_intercept=False, n_jobs=7), \
#                      base_models=[LinearRegression, Ridge],
#                      is_TimeSeries=False)#,\
# ensembleObj.fit(X_train=X_train, y_train=y_train)
# pred=ensembleObj.predict(X_test=X_test)
# r2_score(y_test, pred)

0.96159442778339299